<a href="https://colab.research.google.com/github/m-mehabadi/grad-maker/blob/main/_notebooks/MIDOG_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import torch
# import torchvision

# from torchvision.models.detection import retinanet_resnet50_fpn
# from torchvision.ops import sigmoid_focal_loss
# from torch.optim import Adam

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

!pip install -U plotly

import json
from pathlib import Path
# import plotly
# import plotly.graph_objects as go
# import plotly.express as px
# from plotly.subplots import make_subplots
from tqdm import tqdm
import pandas as pd
import random
import cv2

!apt-get install python3-openslide
from openslide import open_slide

!pip install -U object-detection-fastai

from object_detection_fastai.helper.wsi_loader import *

     |████████████████████████████████| 28.8 MB 2.1 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  javascript-common libjs-jquery libopenslide0 python-asn1crypto
  python-blinker python-cffi-backend python-click python-colorama
  python-cryptography python-enum34 python-flask python-idna python-ipaddress
  python-itsdangerous python-jinja2 python-markupsafe
  python-openslide-examples python-openssl python-pkg-resources
  python-pyinotify python-simplejson python-six python-werkzeug
  python3-olefile python3-pil
Suggested packages:
  apache2 | lighttpd | httpd python-blinker-doc python-cryptography-doc
  python-cryptography-vectors python-enum34-doc python-flask-doc
  python-jinja2-doc python-openssl-doc python-openssl-dbg pytho

In [ ]:
from google.colab import drive
drive.mount('/drive')

folder = "MyDrive/MIDOG_Challenge" #param {type:"string"}
midog_folder = Path("/drive") / Path(folder)

Mounted at /drive


In [ ]:
annotation_file = midog_folder / "MIDOG.json"
image_folder = midog_folder / "images"

hamamatsu_rx_ids = list(range(0, 51))
hamamatsu_360_ids = list(range(51, 101))
aperio_ids = list(range(101, 151))
leica_ids = list(range(151, 201))

rows = []
with open(annotation_file) as f:
    data = json.load(f)

    #categories = {cat["id"]: cat["name"] for cat in data["categories"]}
    categories = {1: 'mitotic figure', 2: 'hard negative'}

    for row in data["images"]:
        file_name = row["file_name"]
        image_id = row["id"]
        width = row["width"]
        height = row["height"]

        scanner  = "Hamamatsu XR"
        if image_id in hamamatsu_360_ids:
            scanner  = "Hamamatsu S360"
        if image_id in aperio_ids:
            scanner  = "Aperio CS"
        if image_id in leica_ids:
            scanner  = "Leica GT450"
         
        for annotation in [anno for anno in data['annotations'] if anno["image_id"] == image_id]:
            box = annotation["bbox"]
            cat = categories[annotation["category_id"]]

            rows.append([file_name, image_id, width, height, box, cat, scanner])

df = pd.DataFrame(rows, columns=["file_name", "image_id", "width", "height", "box", "cat", "scanner"])
df.head()

,file_name,image_id,width,height,box,cat,scanner
0,001.tiff,1,7215,5412,"[4336, 346, 4386, 396]",hard negative,Hamamatsu XR
1,001.tiff,1,7215,5412,"[756, 872, 806, 922]",hard negative,Hamamatsu XR
2,001.tiff,1,7215,5412,"[270, 4044, 320, 4094]",hard negative,Hamamatsu XR
3,001.tiff,1,7215,5412,"[6672.5, 706.5, 6722.5, 756.5]",hard negative,Hamamatsu XR
4,002.tiff,2,7215,5412,"[1872, 319, 1922, 369]",hard negative,Hamamatsu XR


In [ ]:
def sample_function(y, classes, size, level_dimensions, level):
    width, height = level_dimensions[level]
    if len(y[0]) == 0:
        return randint(0, width - size[0]), randint(0, height -size[1])
    else:
        #if randint(0, 5) < 2:
        if True:
            class_id = np.random.choice(classes, 1)[0] # select a random class
            ids = np.array(y[1]) == class_id # filter the annotations according to the selected class
            xmin, ymin, _, _ = np.array(y[0])[ids][randint(0, np.count_nonzero(ids) - 1)] # randomly select one of the filtered annotatons as seed for the training patch
            
            # To have the selected annotation not in the center of the patch and an random offset.
            xmin += random.randint(-size[0]/2, size[0]/2) 
            ymin += random.randint(-size[1]/2, size[1]/2)
            xmin, ymin = max(0, int(xmin - size[0] / 2)), max(0, int(ymin -size[1] / 2))
            xmin, ymin = min(xmin, width - size[0]), min(ymin, height - size[1])
            return xmin, ymin
        else:
            return randint(0, width - size[0]), randint(0, height -size[1])

In [ ]:
def create_wsi_container(annotations_df: pd.DataFrame):

    container = []

    for image_name in tqdm(annotations_df["file_name"].unique()):

        image_annos = annotations_df[annotations_df["file_name"] == image_name]

        bboxes = [box   for box   in image_annos["box"]]
        labels = [label for label in image_annos["cat"]]

        container.append(SlideContainer(image_folder/image_name, y=[bboxes, labels], level=res_level,width=patch_size, height=patch_size, sample_func=sample_function))

    return container

In [ ]:
train_scanner = "Hamamatsu XR" #param ["Hamamatsu XR", "Hamamatsu S360", "Aperio CS"]  {allow-input: true}
train2_scanner = "Aperio CS" #param ["Hamamatsu XR", "Hamamatsu S360", "Aperio CS"]  {allow-input: true}
val_scanner = "Hamamatsu S360" #param ["Hamamatsu XR", "Hamamatsu S360", "Aperio CS"]  {allow-input: true}

patch_size = 256 #param [256, 512, 1024]
res_level = 0

train_annos = df[df["scanner"].isin(train_scanner.split(","))]
train_container = create_wsi_container(train_annos)

train2_annos = df[df["scanner"].isin(train2_scanner.split(","))]
train2_container = create_wsi_container(train2_annos)

val_annos = df[df["scanner"].isin(val_scanner.split(","))]
valid_container = create_wsi_container(val_annos)

f"Created: {len(train_container)} training WSI container and {len(valid_container)} validation WSI container"

100%|██████████| 50/50 [00:22<00:00,  2.24it/s]


'Created: 50 training WSI container and 50 validation WSI container'

In [ ]:
import numpy as np
train_samples_per_scanner = 100 #param {type:"integer"} 1500
val_samples_per_scanner = 50 #param {type:"integer"} 500

train_images = list(np.random.choice(train_container, train_samples_per_scanner))
train2_images = list(np.random.choice(train2_container, train_samples_per_scanner))
valid_images = list(np.random.choice(valid_container, val_samples_per_scanner))

In [ ]:
batch_size = 4 #param {type:"integer"}

#markdown Lets add some basic data [augmentation](https://docs.fast.ai/vision.augment.html)
do_flip = True #param {type:"boolean"}
flip_vert = True #param {type:"boolean"}
max_rotate = 90 #param {type:"number"}
max_zoom = 1.1 #param {type:"number"}
max_lighting = 0.2 #param {type:"number"}
max_warp = 0.2 #param {type:"number"}
p_affine = 0.75 #param {type:"number"}
p_lighting = 0.75 #param {type:"number"}



tfms = get_transforms(do_flip=do_flip,
                      flip_vert=flip_vert,
                      max_rotate=max_rotate,
                      max_zoom=max_zoom,
                      max_lighting=max_lighting,
                      max_warp=max_warp,
                      p_affine=p_affine,
                      p_lighting=p_lighting)

train, train2, valid = ObjectItemListSlide(train_images), ObjectItemListSlide(train2_images), ObjectItemListSlide(valid_images)

class ImageBBox(ImagePoints):
    "Support applying transforms to a `flow` of bounding boxes."
    def __init__(self, flow:FlowField, scale:bool=True, y_first:bool=True, labels:Collection=None,
                 classes:dict=None, pad_idx:int=0):
        super().__init__(flow, scale, y_first)
        self.pad_idx = pad_idx
        if labels is not None and len(labels)>0 and not isinstance(labels[0],Category):
            labels = array([Category(l,classes[l]) for l in labels])
        self.labels = labels

    def clone(self) -> 'ImageBBox':
        "Mimic the behavior of torch.clone for `Image` objects."
        flow = FlowField(self.size, self.flow.flow.clone())
        return self.__class__(flow, scale=False, y_first=False, labels=self.labels, pad_idx=self.pad_idx)

    @classmethod
    def create(cls, h:int, w:int, bboxes:Collection[Collection[int]], labels:Collection=None, classes:dict=None,
               pad_idx:int=0, scale:bool=True)->'ImageBBox':
        "Create an ImageBBox object from `bboxes`."
        if isinstance(bboxes, np.ndarray) and bboxes.dtype == np.object: bboxes = np.array([bb for bb in bboxes])
        bboxes = tensor(bboxes).float()
        tr_corners = torch.cat([bboxes[:,0][:,None], bboxes[:,3][:,None]], 1)
        bl_corners = bboxes[:,1:3].flip(1)
        bboxes = torch.cat([bboxes[:,:2], tr_corners, bl_corners, bboxes[:,2:]], 1)
        flow = FlowField((h,w), bboxes.view(-1,2))
        return cls(flow, labels=labels, classes=classes, pad_idx=pad_idx, y_first=True, scale=scale)

    def _compute_boxes(self) -> Tuple[LongTensor, LongTensor]:
        bboxes = self.flow.flow.flip(1).view(-1, 4, 2).contiguous()
        mins, maxes = bboxes.min(dim=1)[0], bboxes.max(dim=1)[0]
        bboxes = torch.cat([mins, maxes], 1)
        mask = (bboxes[:,2]-bboxes[:,0] > 0) * (bboxes[:,3]-bboxes[:,1] > 0)
        if len(mask) == 0: return tensor([self.pad_idx] * 4), tensor([self.pad_idx])
        res = bboxes[mask]
        if self.labels is None: return res,None
        return res, self.labels[to_np(mask).astype(bool)]

    @property
    def data(self)->Union[FloatTensor, Tuple[FloatTensor,LongTensor]]:
        bboxes,lbls = self._compute_boxes()
        lbls = np.array([o.data for o in lbls]) if lbls is not None else None
        return bboxes if lbls is None else (bboxes, lbls)

    def show(self, y:Image=None, ax:plt.Axes=None, figsize:tuple=(3,3), title:Optional[str]=None, hide_axis:bool=True,
        color:str='white', **kwargs):
        "Show the `ImageBBox` on `ax`."
        if ax is None: _,ax = plt.subplots(figsize=figsize)
        bboxes, lbls = self._compute_boxes()
        h,w = self.flow.size
        bboxes.add_(1).mul_(torch.tensor([h/2, w/2, h/2, w/2])).long()
        for i, bbox in enumerate(bboxes):
            if lbls is not None: text = str(lbls[i])
            else: text=None
            _draw_rect(ax, bb2hw(bbox), text=text, color=color)


class SlideObjectCategoryList(ObjectCategoryList):

    def get(self, i, x: int=0, y: int=0):
        h, w = self.x.items[i].shape
        bboxes, labels = self.items[i]

        bboxes = np.array([box for box in bboxes]) if len(np.array(bboxes).shape) == 1 else  np.array(bboxes)
        labels = np.array(labels)

        if len(labels) > 0:
            bboxes[:, [0, 2]] = bboxes[:, [0, 2]] - x
            bboxes[:, [1, 3]] = bboxes[:, [1, 3]] - y

            bb_widths = (bboxes[:, 2] - bboxes[:, 0]) / 2
            bb_heights = (bboxes[:, 3] - bboxes[:, 1]) / 2

            ids = ((bboxes[:, 0] + bb_widths) > 0) \
                      & ((bboxes[:, 1] + bb_heights) > 0) \
                      & ((bboxes[:, 2] - bb_widths) < w) \
                      & ((bboxes[:, 3] - bb_heights) < h)

            bboxes = bboxes[ids]
            bboxes = np.clip(bboxes, 0, max(h,w))
            bboxes = bboxes[:, [1, 0, 3, 2]]


            labels = labels[ids]
        
        if len(labels) == 0:
            labels = np.array([0])
            bboxes = np.array([[0, 0, patch_size, patch_size]])

        xx = ImageBBox.create(h, w, bboxes, labels, classes=self.classes, pad_idx=self.pad_idx, scale=True)
        # print(xx.data)
        s = torch.tensor([xx.flow.size[0]/2, xx.flow.size[1]/2])[None]
        xx.flow.flow = (xx.flow.flow+1)*s[0][0]
        # print(xx.data)
        # print(xx.data)
        return xx



item_list = ItemLists(".", train, valid)
lls = item_list.label_from_func(lambda x: x.y, label_cls=SlideObjectCategoryList)
lls = lls.transform(tfms, tfm_y=False, size=patch_size)
data = lls.databunch(bs=batch_size, collate_fn=bb_pad_collate,num_workers=0).normalize()

item2_list = ItemLists(".", train2, valid)
lls2 = item2_list.label_from_func(lambda x: x.y, label_cls=SlideObjectCategoryList)
lls2 = lls2.transform(tfms, tfm_y=False, size=patch_size)
data2 = lls2.databunch(bs=batch_size, collate_fn=bb_pad_collate,num_workers=0).normalize()

/usr/local/lib/python3.7/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(a, dtype=dtype, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/fastai/vision/transform.py:247: UserWarning: torch.solve is deprecated in favor of torch.linalg.solveand will be removed in a future PyTorch release.
torch.linalg.solve has its arguments reversed and does not return the LU

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from random import shuffle

def pcgrad(domain_grads):
    """ Projecting conflicting gradients (PCGrad). """
    task_order = list(range(len(domain_grads)))

    # Run tasks in random order
    shuffle(task_order)

    # Initialize task gradients
    grad_pc = [g.clone() for g in domain_grads]

    for i in task_order:

        # Run other tasks
        other_tasks = [j for j in task_order if j != i]

        for j in other_tasks:
            grad_j = domain_grads[j]

            # Compute inner product and check for conflicting gradients
            inner_prod = torch.dot(grad_pc[i], grad_j)
            if inner_prod < 0:
                # Sustract the conflicting component
                grad_pc[i] -= inner_prod / (grad_j ** 2).sum() * grad_j

    # Sum task gradients
    new_grads = torch.stack(grad_pc).sum(0)

    return new_grads

def gradient_maker(domain_grads):
    dgr = torch.stack((domain_grads))

    alpha = 0.01
    epsilon = 100
    number_of_domains, dim = dgr.shape

    g = torch.zeros(dim).to(device)
    u_ = torch.zeros(number_of_domains).to(device)

    for i in range(5):
        u_ = u_ + alpha*(epsilon - (dgr@g))
        g = (1./number_of_domains)*torch.sum(((1+(u_>=0)*u_).view(number_of_domains, 1))*dgr, axis=0)

    return g

In [ ]:
def get_loop_data(data):
  X, y = data

  boxes = map(lambda xx: list(map(lambda yy: np.array([0.,0.,patch_size*1.,patch_size*1.]) if torch.all(yy.eq(torch.tensor([0.,0.,0.,0.]))) else yy, xx)), y[0])
  boxes = list(boxes)
  ys = list(map(
      lambda x: {"boxes": torch.tensor(list(map(lambda xx: xx.tolist(), x[0]))).to(device), "labels": x[1].to(device)},
      zip(boxes, y[1])))
  
  return X.to(device), ys

In [ ]:
import matplotlib.pyplot as plt

def validate(net, loader, criterion=None):
    # iterates the loader and returns loss of the network on the whole loader
    net.eval()
    total_score = 0
    count = 0
    for i, data in enumerate(loader):
        # X, y = data[0].to(device), data[1].to(device)
        # X, y = data
        # y_pred = net(X)
        # loss = criterion(y_pred, y)

        X, y = get_loop_data(data)
        # img = Image.fromarray(data[0][0].detach().numpy(), 'RGB')
        # img.show()

        # plt.imshow(  data[0][0].permute(1, 2, 0)  )
        
        outputs = net(X, y)
        for output in outputs:
            # print(outputs)
            total_score += output['scores'].detach().cpu().numpy().mean().item()
            count += 1
        # print(loss)
        # print("train loss:", loss)

        # classification_loss = loss['classification']
        # regression_loss = loss['bbox_regression']

        # loss_sum = classification_loss.mean() + regression_loss.mean()

        # total_loss += loss.item()
        # correct_predictions += torch.sum(torch.argmax(y_pred, dim=1)==y).item()

    # total_loss /= len(loader)
    # correct_predictions /= (len(loader)*loader.batch_size)

    return total_score/count

In [ ]:
def get_grads(net):
    grads = []
    # for p in self.network.parameters():
    for p in net.parameters():
      try:
        grads.append(p.grad.data.clone().flatten())
      except:
        continue
    # print(grads)
    return torch.cat(grads)

def set_grads(net, new_grads):
    start = 0
    # for k, p in enumerate(self.network.parameters()):
    for k, p in enumerate(net.parameters()):
        dims = p.shape
        end = start + dims.numel()
        # print("new_grads",new_grads.shape)
        p.grad.data = new_grads[start:end].reshape(dims)
        start = end

In [ ]:
import gc

In [ ]:
from os import X_OK
def train_GM(net, opt, criterion, trainloaders, grad_fn, epochs=50, testloader=None):

    make_batches = lambda: zip(*trainloaders)

    for epoch in range(epochs):
        torch.cuda.empty_cache()
        
        for batch_number, domains_batches in enumerate(make_batches()):
            count = 0
            train_loss = 0

            net.train()
            opt.zero_grad()
            domains_grads = []

            with torch.set_grad_enabled(True):
                for domain_number, domain_batch in enumerate(domains_batches):
     
                    X, y = get_loop_data(domain_batch)
                    
                    loss = net(X, y)

                    classification_loss = loss['classification']
                    regression_loss = loss['bbox_regression']


                    total_loss = classification_loss.mean() + regression_loss.mean()
                    total_loss.backward()

                    # print('c', classification_loss.item(), 'r', regression_loss.item(), 't', total_loss.item())
                    count += 1
                    train_loss += total_loss.item()

                    del total_loss
                    del loss


                    domains_grads.append(get_grads(net))
                    opt.zero_grad()
            
            #
            gc.collect()
            
            new_grads = grad_fn(domains_grads)
            set_grads(net, new_grads)
            opt.step()

            train_loss /= count
            print(f'Epoch: {epoch}, Batch number: {batch_number}, Train loss: {train_loss:.5f}')

        if testloader != None:
            train_f1_score = 0. # (validate(net, trainloaders[0]) + validate(net, trainloaders[1]))/2
            val_f1_score = validate(net, testloader)
            print('********************************************************************************')
            print(f'Finished epoch: {epoch}, Training F1-Score: {train_f1_score:.5f}, Validation F1-Score: {val_f1_score:.5f}')
            print('********************************************************************************')

In [ ]:
import torch
import torchvision
from torchvision.models.detection import RetinaNet
from torchvision.models.detection.anchor_utils import AnchorGenerator

# backbone = torchvision.models.resnet50()

# backbone.out_channels = 1280
# anchor_generator = AnchorGenerator(
#     sizes=((32),),
#     aspect_ratios=((1.0),)
# )

# put the pieces together inside a RetinaNet model
# model_GM = RetinaNet(backbone,
#                   num_classes=2,
#                  anchor_generator=anchor_generator)

# https://pytorch.org/vision/stable/_modules/torchvision/models/detection/retinanet.html
# https://pytorch.org/vision/main/generated/torchvision.models.detection.retinanet_resnet50_fpn.html
model_GM = torchvision.models.detection.retinanet_resnet50_fpn(pretrained=False, pretrained_backbone=False)
criterion = torchvision.ops.sigmoid_focal_loss
optimizer = torch.optim.Adam(model_GM.parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=0.0005)

model_GM.to(device)

train_dl1 = data.train_dl.dl
train_dl2 = data2.train_dl.dl

valid_dl = data.valid_dl.dl

In [ ]:
# torch.cuda.empty_cache()

# torch.cuda.memory_summary(device=None, abbreviated=False)

In [ ]:
train_GM(model_GM, optimizer, criterion, [train_dl1, train_dl2], pcgrad, 10, valid_dl)

_, mnist_test_acc = validate(model_GM, valid_dl, criterion)
# _, svhn_test_acc = validate(model_GM, svhn_testloader, criterion)

print(f'Test Acc: {mnist_test_acc:.5f}')
        # f', SVHN Test Acc: {svhn_test_acc:.5f}')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Epoch: 0, Batch number: 0, Train loss: 2.16417
Epoch: 0, Batch number: 1, Train loss: 28329.26270
Epoch: 0, Batch number: 2, Train loss: 2.00083
Epoch: 0, Batch number: 3, Train loss: 38.08365
Epoch: 0, Batch number: 4, Train loss: 2.25905
Epoch: 0, Batch number: 5, Train loss: 2.65457
Epoch: 0, Batch number: 6, Train loss: 2.43708
Epoch: 0, Batch number: 7, Train loss: 2.57926
Epoch: 0, Batch number: 8, Train loss: 2.74641
Epoch: 0, Batch number: 9, Train loss: 3.02642
Epoch: 0, Batch number: 10, Train loss: 5.09432
Epoch: 0, Batch number: 11, Train loss: 5.45908
Epoch: 0, Batch number: 12, Train loss: 3.20125
Epoch: 0, Batch number: 13, Train loss: 3.90113
Epoch: 0, Batch number: 14, Train loss: 2.76156
Epoch: 0, Batch number: 15, Train loss: 2.71140
Epoch: 0, Batch number: 16, Train loss: 2.50702
Epoch: 0, Batch number: 17, Train loss: 3.61308
Epoch: 0, Batch number: 18, Train loss: 4.51286
Epoch: 0, Batch number: 19, Train loss: 2.52922
Epoch: 0, Batch number: 20, Train loss: 2.147

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: Mean of empty slice.
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


********************************************************************************
Finished epoch: 0, Training F1-Score: 0.00000, Validation F1-Score: nan
********************************************************************************
Epoch: 1, Batch number: 0, Train loss: 1.55335
Epoch: 1, Batch number: 1, Train loss: 1.71405
Epoch: 1, Batch number: 2, Train loss: 1.65776
Epoch: 1, Batch number: 3, Train loss: 1.56707
Epoch: 1, Batch number: 4, Train loss: 1.68661
Epoch: 1, Batch number: 5, Train loss: 1.57085
Epoch: 1, Batch number: 6, Train loss: 1.65160
Epoch: 1, Batch number: 7, Train loss: 1.44983
Epoch: 1, Batch number: 8, Train loss: 1.44038
Epoch: 1, Batch number: 9, Train loss: 1.55160
Epoch: 1, Batch number: 10, Train loss: 1.33733
Epoch: 1, Batch number: 11, Train loss: 1.34685
Epoch: 1, Batch number: 12, Train loss: 1.90033
Epoch: 1, Batch number: 13, Train loss: 1.58320
Epoch: 1, Batch number: 14, Train loss: 1.68070
Epoch: 1, Batch number: 15, Train loss: 1.61441
Epoch: 1

TypeError: ignored

In [ ]:
_, mnist_test_acc = validate(model_GM, valid_dl, criterion)
# _, svhn_test_acc = validate(model_GM, svhn_testloader, criterion)

print(f'Test Acc: {mnist_test_acc:.5f}')

In [ ]:
for i, data in enumerate(valid_dl):
        X, y = get_loop_data(data)
        output = model_GM(X)
        print(output)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


[{'boxes': tensor([[ 69.8425, 148.5489,  97.7951, 176.8014],
        [ 59.5955, 148.5371,  87.5594, 176.7938],
        [ 80.0242, 148.5543, 108.0110, 176.8428],
        [ 49.3207, 148.5326,  77.3133, 176.8146],
        [ 74.8074, 158.7111, 102.8589, 187.0270],
        [ 39.0393, 148.5282,  67.0630, 176.8392],
        [ 64.5624, 158.6945,  92.6249, 187.0165],
        [ 85.0027, 158.7247, 113.0805, 187.0681],
        [ 64.6776, 138.3693,  92.6873, 166.6986],
        [ 28.7652, 148.5212,  56.8140, 176.8535],
        [ 69.8425, 148.5489,  97.7951, 176.8014],
        [ 90.1429, 148.5481, 118.2129, 176.9131],
        [ 54.3055, 158.6975,  82.3825, 187.0351],
        [ 54.4232, 138.3561,  82.4474, 166.6920],
        [ 74.8658, 138.3766, 102.9029, 166.7351],
        [ 44.0514, 158.7131,  72.1400, 187.0657],
        [ 59.5955, 148.5371,  87.5594, 176.7938],
        [ 18.5023, 148.5267,  46.5675, 176.8738],
        [ 44.1444, 138.3456,  72.1957, 166.7001],
        [ 79.9912, 153.6388, 107.9945, 

In [ ]:
x = [[5,6,7,8], [1, 2, 3, 4]]
y = torch.tensor(x)
y.tolist()